In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. 패키지 설치 및 데이터 불러오기

In [ ]:
!pip install pytorch_pretrained_bert==0.4.0
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
# 토크나이저 mecab 설치
! bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 75 (delta 33), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [ ]:
cd /content/drive/MyDrive/마이야르/6.Code 

/content/drive/.shortcut-targets-by-id/1enR-TKNggkQ_bAdN7UF7isfRkeewUc_-/NH_dacon/태욱


In [ ]:
from __future__ import absolute_import, division, print_function

import argparse
import csv
import logging
import os
import random
import sys
import warnings
import pickle
warnings.filterwarnings("ignore")

from collections import Counter
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange, tqdm_notebook, notebook

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
### kyoungman.bae @ 19-05-28 
# from tokenization_morp import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear

import urllib3
import json

from konlpy.tag import Mecab
# from khaiii import KhaiiiApi
# import kss

mecab = Mecab()
# khaiii = KhaiiiApi()

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/마이야르/1.Data/news_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/마이야르/1.Data/news_test.csv")
submission = pd.read_csv("/content/drive/MyDrive/마이야르/1.Data/sample_submission.csv")
train_df.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


## 2. 전처리 및 모델 세팅

In [ ]:
train, validation, train_labels, validation_labels = train_test_split(train_df.iloc[:,:-1], train_df['info'], 
                                                                   random_state=2020, test_size=0.2)

train = pd.concat([train,train_labels],axis=1)
validation = pd.concat([validation,validation_labels],axis=1)

train.to_csv('/content/drive/MyDrive/마이야르/1.Data/train.csv', index=False)
validation.to_csv('/content/drive/MyDrive/마이야르/1.Data/dev.csv', index=False)

In [ ]:
def construct_vocab(file_, max_size=200000, mincount=5):
    vocab2id = {'[CLS]': 2, '[SEP]': 3, '[PAD]': 0, '[UNK]': 1, '[STOP]': 4}
    # id2vocab = {2: '[CLS]', 3: '[SEP]', 0: '[PAD]', 1: '[UNK]', 4: '[STOP]'}
    word_pad = {'[CLS]': 2, '[SEP]': 3, '[PAD]': 0, '[UNK]': 1, '[STOP]': 4}
    
    cnt = len(vocab2id)
    with open(file_, 'r') as fp:
      for line in fp:
        arr = re.split('\t', line[:-1])
        if (arr[0] in [' ', 'n_iters=10000', 'max_length=16', '[MASK]','<S>','<T>']) :
          continue
        if arr[0] in word_pad:
          continue
        if int(arr[1]) >= mincount:
          vocab2id[arr[0]] = cnt
          cnt += 1
        if len(vocab2id) == max_size:
          break
  
    return vocab2id

vocab2id = construct_vocab('/content/drive/MyDrive/마이야르/4.pre_trained embedding/korbert/vocab.korean_morp.list')

In [ ]:
## abstract data
def mecab_tokenizer(x):
  
  #sntc_list = kss.split_sentences(x)
  tk = []

  token = mecab.pos(x)

  #print(token)
  for t in token:
    if t[1] == 'NNBC':
      #print(t[0] + "/" + t[1] + "_")
      tk += [t[0] + "/" + "NNB" + "_"]
    elif  (t[1] == 'SSO') or (t[1] == 'SSC'):
      #print(t[0] + "/" + t[1] + "_")
      tk += [t[0] + "/" + "SS" + "_"]
    elif t[1] == 'SY':
      #print(t[0] + "/" + t[1] + "_")
      tk += [t[0] + "/" + "SW" + "_"]
    else: 
      #print(t[0] + "/" + t[1] + "_")
      tk += [t[0] + "/" + t[1] + "_"]

  return tk

In [ ]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
import easydict 
args = easydict.EasyDict({"data_dir": '/content/drive/MyDrive/마이야르/1.Data',
                          "task_name":'cola',
                          "output_dir":'/content/drive/MyDrive/마이야르/5.Model/bert_tu',
                          "vocab_file":'/content/drive/MyDrive/마이야르/4.pre_trained embedding/korbert/vocab.korean_morp.list',
                          "openapi_key":'',
                          "bert_model_path":'/content/drive/MyDrive/마이야르/4.pre_trained embedding/korbert',
                          "cache_dir":'',
                          "max_seq_length":200,
                          "do_train":True,
                          "do_eval":False,
                          "do_lower_case":False,
                          "train_batch_size":16,
                          "eval_batch_size":16,
                          "learning_rate":2e-5,
                          "num_train_epochs":5,
                          "warmup_proportion":0.1,
                          "no_cuda": False,
                          "local_rank":-1,
                          "seed":42,
                          "gradient_accumulation_steps":1,
                          "fp16":False,
                          "loss_scale":0,
                          "server_ip":'',
                          "server_port":''    
})

In [ ]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""
    
    def __init__(self, input_ids, input_mask, segment_ids, label_id, sentence_id=None):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()
    
    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()


    def _read_csv(self,input_file):
        return pd.read_csv(input_file)


In [ ]:
class ColaProcessor(DataProcessor):
    """Processor for the CoLA data set (GLUE version)."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "train.csv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "dev.csv")), "dev")
  
    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "news_test.csv")), "test")

    def get_labels(self, data_dir):
        """See base class."""
        ######################################################################
        ### kyoungman.bae @ 19-05-30 @ for multi label classification
        ### You need to add a file with label information in the data folder. 
        ### You should use a numbered label on a line.
        labels =[]
        lines = self._read_tsv(os.path.join(data_dir, "labels.tsv"))
        for (i, line) in enumerate(lines):
            labels.append(str(line[0]))   
        return labels
       # return ["0", "1"]

    # def _create_examples(self, lines, set_type):
    #     """Creates examples for the training and dev sets."""
    #     examples = []
    #     for (i, line) in enumerate(lines):
    #         guid = "%s-%s" % (set_type, i)
    #         text_a = line[3]
    #         label = line[5]
    #         examples.append(
    #             InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    #     return examples
    
    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for i in range(len(lines)):
          guid = "%s-%s" % (set_type, i)
          text_a = lines.loc[i,'title']
          try:
            label = lines.loc[i,'info']
          except:
            label = 0
          text_b = lines.loc[i,'content']
          examples.append(
              InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

### kyoungman.bae @ 19-05-28 @ 
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, openapi_key):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}
    ### kyoungman.bae @ 19-05-28 @
    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = ""
        tokens_b = ""

        tokens_a = tokenizer(example.text_a)    
        tokens_b = tokenizer(example.text_b)

        # tokens_b = None
        if example.text_b:

        #     # Modifies `tokens_a` and `tokens_b` in place so that the total
        #     # length is less than the specified length.
        #     # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
        #     # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length-2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        if len(tokens_a) > (max_seq_length - 2):
          tokens_a = tokens_a[:max_seq_length - 2]

        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = [vocab2id[i] if i in vocab2id else vocab2id['[UNK]'] for i in tokens] 

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding
 

     
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        #print(ex_index)        
        label_id = label_map[str(example.label)]
        
        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (str(example.label), label_id))
        senid = example.guid+""
        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features
    
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)


In [ ]:
processors = {
    "cola": ColaProcessor
}

num_labels_task = {
    "cola": 2
}

if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
else:
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')

logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
    device, n_gpu, bool(args.local_rank != -1), args.fp16))

if args.gradient_accumulation_steps < 1:
    raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                        args.gradient_accumulation_steps))

args.train_batch_size = args.train_batch_size // args.gradient_accumulation_steps

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

if not args.do_train and not args.do_eval:
    raise ValueError("At least one of `do_train` or `do_eval` must be True.")
### kyoungman.bae @ 19-05-28
if args.openapi_key == None:
    raise ValueError("If you use the POS-based BPE, you must insert the access key.")

if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train:
    raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

task_name = args.task_name.lower()

if task_name not in processors:
    raise ValueError("Task not found: %s" % (task_name))

processor = processors[task_name]()
### kyoungman.bae @ 19-05-28 

label_list = ['0','1']
num_labels = len(label_list)

### kyoungman.bae @ 19-05-28
tokenizer = mecab_tokenizer

train_examples = None
num_train_optimization_steps = None
if args.do_train:
    train_examples = processor.get_train_examples(args.data_dir)
    num_train_optimization_steps = int(
        len(train_examples) / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs
    if args.local_rank != -1:
        num_train_optimization_steps = num_train_optimization_steps // torch.distributed.get_world_size()

# Prepare model
cache_dir = args.cache_dir if args.cache_dir else os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed_{}'.format(args.local_rank))
model = BertForSequenceClassification.from_pretrained(args.bert_model_path,
          cache_dir=cache_dir,
          num_labels = num_labels)
if args.fp16:
    model.half()
model.to(device)
if args.local_rank != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    model = DDP(model)
elif n_gpu > 1:
    model = torch.nn.DataParallel(model)

# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
if args.fp16:
    try:
        from apex.optimizers import FP16_Optimizer
        from apex.optimizers import FusedAdam
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    optimizer = FusedAdam(optimizer_grouped_parameters,
                          lr=args.learning_rate,
                          bias_correction=False,
                          max_grad_norm=1.0)
    if args.loss_scale == 0:
        optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
    else:
        optimizer = FP16_Optimizer(optimizer, static_loss_scale=args.loss_scale)

else:
    optimizer = BertAdam(optimizer_grouped_parameters,
                          lr=args.learning_rate,
                          warmup=args.warmup_proportion,
                          t_total=num_train_optimization_steps)



## 3.Train

In [ ]:
eval_examples = processor.get_dev_examples(args.data_dir)
eval_features = convert_examples_to_features(
    eval_examples, label_list, args.max_seq_length, tokenizer,args.openapi_key)

all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
del all_input_ids, all_input_mask, all_segment_ids, all_label_ids

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)


global_step = 0
nb_tr_steps = 0
tr_loss = 0

if args.do_train:
    ### kyoungman.bae @ 19-05-30 @ do_anal_morp, openapi_accessKey 인수 추가
    train_features = convert_examples_to_features(
        train_examples, label_list, args.max_seq_length, tokenizer, args.openapi_key)                 
    if len(train_features) == 0:
        logger.info("The number of train_features is zero. Please check the tokenization. ")
        sys.exit()

    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_examples))
    logger.info("  Batch size = %d", args.train_batch_size)
    logger.info("  Num steps = %d", num_train_optimization_steps)
    all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
    train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    del all_input_ids, all_input_mask, all_segment_ids, all_label_ids
    if args.local_rank == -1:
        train_sampler = RandomSampler(train_data)
    else:
        train_sampler = DistributedSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

    model.train()
    epoch=0
    for _ in trange(int(args.num_train_epochs), desc="Epoch"):
#        train_logits = []
        
        epoch+=1
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(notebook.tqdm(train_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch
            loss = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask)

            #train_logits.append(logits)

            logits = logits.detach().cpu().numpy()
            label_ids = label_ids.to('cpu').numpy()

            if n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu.
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                optimizer.backward(loss)
            else:
                loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    # modify learning rate with special warm up BERT uses
                    # if args.fp16 is False, BertAdam is used that handles this automatically
                    lr_this_step = args.learning_rate * warmup_linear(global_step/num_train_optimization_steps, args.warmup_proportion)
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = lr_this_step
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        outputs = []
        ##################################################
        ### kyoungman.bae @ 19-05-31
        ### The classified result labels are displayed in the following path.
        output_eval_file = os.path.join(args.output_dir, "eval_results_labels"+str(epoch)+".txt")
        with open(output_eval_file, "w") as writer:
        
            for input_ids, input_mask, segment_ids, label_ids in notebook.tqdm(eval_dataloader, desc="Evaluating"):
                input_ids = input_ids.to(device)
                input_mask = input_mask.to(device)
                segment_ids = segment_ids.to(device)
                label_ids = label_ids.to(device)

                with torch.no_grad():
                    tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
                    logits = model(input_ids, segment_ids, input_mask)
                # val_logits.append(logits)

                logits = logits.detach().cpu().numpy()
                label_ids = label_ids.to('cpu').numpy()
                ### kyoungman.bae @ 19-05-3
                current_out = np.argmax(logits, axis=1)
                for key in current_out:
                    writer.write("%s\n" % (int(key)))

                tmp_eval_accuracy = accuracy(logits, label_ids)

                eval_loss += tmp_eval_loss.mean().item()
                eval_accuracy += tmp_eval_accuracy

                nb_eval_examples += input_ids.size(0)
                nb_eval_steps += 1

        eval_loss = eval_loss / nb_eval_steps
        eval_accuracy = eval_accuracy / nb_eval_examples
        loss = tr_loss/nb_tr_steps if args.do_train else None
        result = {'eval_loss': eval_loss,
                  'eval_accuracy': eval_accuracy,
                  'global_step': global_step,
                  'loss': loss}
        

        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key]))) 


        print('Epoch = {}, loss = {}, val_loss = {}'.format(epoch, tr_loss/len(train_dataloader),eval_loss))
    # Save a trained model and the associated configuration
    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    output_model_file = os.path.join(args.output_dir, WEIGHTS_NAME)
    torch.save(model_to_save.state_dict(), output_model_file)
    output_config_file = os.path.join(args.output_dir, CONFIG_NAME)
    with open(output_config_file, 'w') as f:
        f.write(model_to_save.config.to_json_string())

    # Load a trained model and config that you have fine-tuned
    config = BertConfig(output_config_file)
    model = BertForSequenceClassification(config, num_labels=num_labels)
    model.load_state_dict(torch.load(output_model_file))
else:
    model = BertForSequenceClassification.from_pretrained(args.bert_model_path, num_labels=num_labels)
model.to(device)

12/31/2020 19:31:53 - INFO - __main__ -   *** Example ***
12/31/2020 19:31:53 - INFO - __main__ -   guid: dev-0
12/31/2020 19:31:53 - INFO - __main__ -   tokens: [CLS] 한국/NNP_ 기업/NNG_ 평가/NNG_ (/SS_ 034950/SN_ )/SS_ ,/SC_ 52/SN_ 주/NNB_ 신고/NNG_ 가/JKS_ [SEP] 2020/SN_ 年/NNG_ 상반기/NNG_ "/SW_ 정부/NNG_ 정책/NNG_ "/SW_ 최대/NNG_ 수/NNG_ 혜주/NNG_ TOP/SL_ 10/SN_ 긴급/NNG_ 공개/NNG_ ./SF_ ./SF_ [SEP]
12/31/2020 19:31:53 - INFO - __main__ -   input_ids: 2 98 381 481 27 1 28 1 3191 2225 1380 22 3 6070 1 1 1 176 640 1 721 158 1 1 111 3178 271 5 5 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/31/2020 19:31:53 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0

12/31/2020 20:11:56 - INFO - __main__ -   ***** Eval results *****
12/31/2020 20:11:56 - INFO - __main__ -     eval_accuracy = 0.9863152132721378
12/31/2020 20:11:56 - INFO - __main__ -     eval_loss = 0.04056818325316578
12/31/2020 20:11:56 - INFO - __main__ -     global_step = 5938
12/31/2020 20:11:56 - INFO - __main__ -     loss = 0.09008879510329292
Epoch:  20%|██        | 1/5 [39:33<2:38:12, 2373.20s/it]


Epoch = 1, loss = 0.09008879510329292, val_loss = 0.04056818325316578


12/31/2020 20:50:54 - INFO - __main__ -   ***** Eval results *****
12/31/2020 20:50:54 - INFO - __main__ -     eval_accuracy = 0.9916206998189397
12/31/2020 20:50:54 - INFO - __main__ -     eval_loss = 0.03584455918841468
12/31/2020 20:50:54 - INFO - __main__ -     global_step = 11876
12/31/2020 20:50:54 - INFO - __main__ -     loss = 0.02049854192853788
Epoch:  40%|████      | 2/5 [1:18:31<1:58:07, 2362.66s/it]


Epoch = 2, loss = 0.02049854192853788, val_loss = 0.03584455918841468


...

12/31/2020 22:08:48 - INFO - __main__ -   ***** Eval results *****
12/31/2020 22:08:48 - INFO - __main__ -     eval_accuracy = 0.9928418038654259
12/31/2020 22:08:48 - INFO - __main__ -     eval_loss = 0.03791232860711416
12/31/2020 22:08:48 - INFO - __main__ -     global_step = 23752
12/31/2020 22:08:48 - INFO - __main__ -     loss = 0.0013210052972689186
Epoch:  80%|████████  | 4/5 [2:36:24<39:09, 2349.36s/it]  


Epoch = 4, loss = 0.0013210052972689186, val_loss = 0.03791232860711416


12/31/2020 22:47:43 - INFO - __main__ -   ***** Eval results *****
12/31/2020 22:47:43 - INFO - __main__ -     eval_accuracy = 0.9934313023706262
12/31/2020 22:47:43 - INFO - __main__ -     eval_loss = 0.039735842211325915
12/31/2020 22:47:43 - INFO - __main__ -     global_step = 29690
12/31/2020 22:47:43 - INFO - __main__ -     loss = 0.00038581961663920706
Epoch: 100%|██████████| 5/5 [3:15:19<00:00, 2343.91s/it]


Epoch = 5, loss = 0.00038581961663920706, val_loss = 0.039735842211325915


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30349, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
    

## 4. Inference

In [ ]:
%%time
test_examples = processor.get_test_examples(test_df)
test_features = convert_examples_to_features(
    test_examples, label_list, args.max_seq_length, tokenizer,vocab2id)
logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(test_examples))
logger.info("  Batch size = %d", args.eval_batch_size)
all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)

model.eval()
outputs = []

for input_ids, input_mask, segment_ids, in notebook.tqdm(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)


    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask)

    logits = logits.detach().cpu().numpy()
    current_out = np.argmax(logits, axis=1)
    outputs.extend(current_out)


In [ ]:
Counter(outputs)

In [ ]:
submission['info'] = outputs
submission

,id,info
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1


In [ ]:
train_unique_ad_sentence = train_df.query('info == "1"')['content'].unique()
test_content = test_df['content'].values
ad_index = []
for idx, sent in enumerate(notebook.tqdm(test_content)) : #Test 데이터에 있는 모든 content들에 대하여

    if sent in train_unique_ad_sentence: # Train 데이터의 광고성 문구와 같은지 비교
        submission['info'].iloc[idx] = 1 # 같으면 1
        ad_index.append(idx)
    else : 
        pass 

In [ ]:
submission['info'] = submission['info'].astype('int')
submission['info'].value_counts()

0    89129
1    53436
Name: info, dtype: int64

In [ ]:
submission.to_csv('/content/submission.csv')